In [1]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from numpy import load
from torch.utils.data import Dataset

In [2]:
# load array
Data = load('Data_09.npy')
# print the array

In [3]:
Data=np.nan_to_num(Data)

In [4]:
Xdata=[]
Ydata=[]
Xtrain_data=[]
Ytrain_data=[]
Xtest_data=[]
Ytest_data=[]



for idx in range(len(Data)//39):
    if(np.isinf(Data[39*idx:39*(idx+1)][:,:].tolist()).any()):
        print(np.isinf(Data[39*idx:39*(idx+1)][:,:].tolist()).any())
        continue
    if idx< (len(Data)//39)*0.9:
        Xtrain_data.append(Data[39*idx:39*(idx+1)][:,:].tolist())
        Ytrain_data.append(Data[39*idx:39*(idx+1)][:,-1].tolist())
    else:
        Xtest_data.append(Data[39*idx:39*(idx+1)][:,:].tolist())
        Ytest_data.append(Data[39*idx:39*(idx+1)][:,-1].tolist())
Xtrain_data=np.vstack(Xtrain_data)
Ytrain_data=np.vstack(Ytrain_data)

Xtrain_data=torch.FloatTensor(Xtrain_data)
Ytrain_data=torch.IntTensor(Ytrain_data)
Ytrain_data=Ytrain_data.view(-1)


Xtest_data=np.vstack(Xtest_data)
Ytest_data=np.vstack(Ytest_data)
Xtest_data=torch.FloatTensor(Xtest_data)
Ytest_data=torch.IntTensor(Ytest_data)
Ytest_data=Ytest_data.view(-1)

In [5]:
Ytrain_data=2*(Ytrain_data>0).long()+(Ytrain_data==0).long()
Ytest_data=2*(Ytest_data>0).long()+(Ytest_data==0).long()

Ydata=(torch.stack([Ydata>0,Ydata==0, Ydata<0],axis=0)).float()

In [6]:
Ytrain_data=Ytrain_data.T
Ytest_data=Ytest_data.T

from torch.utils.data import DataLoader, random_split
Xtrain_data,Xtest_data=random_split(Xdata, [157700-15770, 15770])
Ytrain_data,Ytest_data=random_split(Ydata, [157700-15770, 15770])

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    # 데이터셋을 bsz 파트들로 나눕니다.
    nbatch = data.size(0) // bsz
    # 깔끔하게 나누어 떨어지지 않는 추가적인 부분(나머지들) 은 잘라냅니다.
    data = data.narrow(0, 0, nbatch * bsz)
    # 데이터에 대하여 bsz 배치들로 동등하게 나눕니다.
    data = data.view(bsz, -1,data.size(1)).transpose(0,1).contiguous()
    return data.to(device)

In [8]:
bptt = 39
def get_batch(source, i,bs):
    seq_len = min(bptt*bs, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [9]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
#         self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.src_tok_emb = nn.Linear(src_vocab_size, emb_size)
#         self.tgt_tok_emb = nn.Linear(3, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

#     def forward(self,
#                 src: Tensor,
#                 trg: Tensor,
#                 src_mask: Tensor,
#                 tgt_mask: Tensor,
#                 src_padding_mask: Tensor,
#                 tgt_padding_mask: Tensor,
#                 memory_key_padding_mask: Tensor):
    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
#         tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
#         src_emb = self.positional_encoding(src)
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg).squeeze())
#         outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
#                                 src_padding_mask, tgt_padding_mask, memory_key_padding_mask)


        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [10]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)
    src_mask = generate_square_subsequent_mask(src_seq_len)
    return src_mask, tgt_mask

In [11]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = 7
TGT_VOCAB_SIZE = 3
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 10
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

# from torch.nn.utils.rnn import pad_sequence

# # helper function to club together sequential operations
# def sequential_transforms(*transforms):
#     def func(txt_input):
#         for transform in transforms:
#             txt_input = transform(txt_input)
#         return txt_input
#     return func

# # function to collate data samples into batch tesors
# def collate_fn(batch):
#     src_batch, tgt_batch = [], []
#     for src_sample, tgt_sample in batch:
#         src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
#         tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

#     src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
#     tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
#     return src_batch, tgt_batch

In [12]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0

    for batch, i in enumerate(range(0, Xtrain_data.size(0) - 1, BATCH_SIZE*bptt)):
        data, org_targets = get_batch(Xtrain_data, i,BATCH_SIZE)
        if (data.isnan().any() or data.isinf().any()):
#            print(data)
            continue
#         _,targets = get_batch(Ytrain_data,i)
        targets,_ = get_batch(Ytrain_data,i,BATCH_SIZE)

        targets=torch.unsqueeze(targets,1)
        src=batchify(data,BATCH_SIZE)
        tgt=batchify(targets,BATCH_SIZE)
        src_input=src[:-1]
        tgt_input = tgt[1:]


        src_mask, tgt_mask = create_mask(src_input, tgt_input)
        logits = model(src_input, tgt_input, src_mask, tgt_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
        
    return losses / Xtrain_data.size(0)

In [13]:
def evaluate(model):
    model.eval()
    losses = 0
    total=0
    correct=0
    correct1=0
    tot1=0
    correct2=0
    tot2=0
    correct0=0
    tot0=0
    with torch.no_grad():
        for i in range(0, Xtest_data.size(0) - 1, bptt):

            data, org_targets = get_batch(Xtest_data, i,BATCH_SIZE)
            if (data.isnan().any() or data.isinf().any()):
                continue
            targets,_ = get_batch(Ytest_data,i,BATCH_SIZE)
            targets=torch.unsqueeze(targets,1)
            src=batchify(data,BATCH_SIZE)
            tgt=batchify(targets,BATCH_SIZE)

            src_input=src[:-1]
            tgt_input = tgt[1:]
            
            tgt_input = tgt[:-1]
#             print(src_input.size())
#             print(tgt_input.size())
            src_mask, tgt_mask = create_mask(src_input, tgt_input)

#             logits = model(src_input, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
            logits = model(src_input, tgt_input, src_mask, tgt_mask)

            
            tgt_out = tgt[1:]

            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
            
            
            if(loss.isnan()):
                print(src,tgt_input)
                break
            losses += loss.item()
            _,predicted=torch.max(logits,-1)
            
            correct += (tgt_out.squeeze() == predicted).sum().item()
            total+=len(predicted)*BATCH_SIZE
            tot0+=(0== tgt_out.squeeze()).sum().item()
            tot1+=(1== tgt_out.squeeze()).sum().item()
            tot2+=(2== tgt_out.squeeze()).sum().item()
            correct0+=((0== predicted) &(0==tgt_out.squeeze())).sum().item()
            correct1+=((1== predicted)&(1==tgt_out.squeeze())).sum().item()
            correct2+=((2== predicted)&(2==tgt_out.squeeze())).sum().item()
            
    print(total)
    print(correct)
    print("Acc:",correct/total*100)            
    print("Acc0:",correct0/tot0*100,correct0,tot0)     
    print("Acc1:",correct1/tot1*100,correct1,tot1)     
    print("Acc2:",correct2/tot2*100,correct2,tot2)     
    return losses / Xtest_data.size(0)

In [14]:
evaluate(transformer)

168430
34865
Acc: 20.699994062815414
Acc0: 82.42776562679866 28642 34748
Acc1: 2.208449798622956 2473 111979
Acc2: 17.278717228032992 3750 21703


0.035481424241466615

In [15]:
from timeit import default_timer as timer
NUM_EPOCHS = 200
best_val_loss=100000000
for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = transformer
PATH='best_model_seq'
torch.save(best_model.state_dict(), PATH)


168430
149638
Acc: 88.84284272398028
Acc0: 81.29676528145505 28249 34748
Acc1: 93.3648273336965 104549 111979
Acc2: 77.59295949868682 16840 21703
Epoch: 1, Train loss: 0.000, Val loss: 0.048, Epoch time = 14.037s
168430
149638
Acc: 88.84284272398028
Acc0: 81.29676528145505 28249 34748
Acc1: 93.3648273336965 104549 111979
Acc2: 77.59295949868682 16840 21703
Epoch: 2, Train loss: 0.000, Val loss: 0.063, Epoch time = 13.872s
168430
149638
Acc: 88.84284272398028
Acc0: 81.29676528145505 28249 34748
Acc1: 93.3648273336965 104549 111979
Acc2: 77.59295949868682 16840 21703
Epoch: 3, Train loss: 0.000, Val loss: 0.072, Epoch time = 13.876s
168430
149638
Acc: 88.84284272398028
Acc0: 81.29676528145505 28249 34748
Acc1: 93.3648273336965 104549 111979
Acc2: 77.59295949868682 16840 21703
Epoch: 4, Train loss: 0.000, Val loss: 0.070, Epoch time = 13.934s
168430
149638
Acc: 88.84284272398028
Acc0: 81.29676528145505 28249 34748
Acc1: 93.3648273336965 104549 111979
Acc2: 77.59295949868682 16840 21703
Ep

168430
149638
Acc: 88.84284272398028
Acc0: 81.29676528145505 28249 34748
Acc1: 93.3648273336965 104549 111979
Acc2: 77.59295949868682 16840 21703
Epoch: 40, Train loss: 0.000, Val loss: 0.069, Epoch time = 13.865s
168430
149638
Acc: 88.84284272398028
Acc0: 81.29676528145505 28249 34748
Acc1: 93.3648273336965 104549 111979
Acc2: 77.59295949868682 16840 21703
Epoch: 41, Train loss: 0.000, Val loss: 0.061, Epoch time = 13.928s
168430
149638
Acc: 88.84284272398028
Acc0: 81.29676528145505 28249 34748
Acc1: 93.3648273336965 104549 111979
Acc2: 77.59295949868682 16840 21703
Epoch: 42, Train loss: 0.000, Val loss: 0.060, Epoch time = 13.919s
168430
149638
Acc: 88.84284272398028
Acc0: 81.29676528145505 28249 34748
Acc1: 93.3648273336965 104549 111979
Acc2: 77.59295949868682 16840 21703
Epoch: 43, Train loss: 0.000, Val loss: 0.068, Epoch time = 13.907s
168430
149638
Acc: 88.84284272398028
Acc0: 81.29676528145505 28249 34748
Acc1: 93.3648273336965 104549 111979
Acc2: 77.59295949868682 16840 2170

KeyboardInterrupt: 